In [14]:
import datetime
from datetime import datetime as dt
import time
import requests
import pandas as pd
import numpy as np
import ta

def unix_to_local_time(unix_timestamp):
    return str(dt.fromtimestamp(unix_timestamp).strftime('%Y-%m-%d %H:%M:%S'))

def trade_option(symbolID, days_before, timeframe):
    end_time_in_millis = int(time.time() * 1000)
    end_time = datetime.datetime.fromtimestamp(end_time_in_millis / 1000)

    # Calculate the start time based on the number of days before the end time
    start_time = end_time - datetime.timedelta(days=days_before)
    start_time = start_time.replace(hour=9, minute=15, second=0, microsecond=0)

    # Convert start and end times to Unix timestamps in milliseconds
    start_time_in_millis = int(start_time.timestamp() * 1000)
    end_time_in_millis = int(end_time.timestamp() * 1000)

    ourl = 'https://groww.in/v1/api/stocks_fo_data/v4/charting_service/chart/exchange/NSE/segment/FNO/'+symbolID
    url = 'https://groww.in/v1/api/charting_service/v4/chart/exchange/NSE/segment/CASH/'+'NIFTY'
    params = {
        'endTimeInMillis': str(end_time_in_millis),
        'intervalInMinutes': timeframe,
        'startTimeInMillis': str(start_time_in_millis),
    }
    
    response = requests.get(ourl, params=params)
    
    if response.status_code == 200:
        json_data = response.json()
        df = pd.DataFrame(json_data)
    
    df.drop(['changeValue', 'changePerc', 'closingPrice', 'startTimeEpochInMillis'], axis=1, inplace=True)
    ohlc = df['candles'].apply(pd.Series)
    
    ohlc.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    ohlc.drop(['volume'], axis=1, inplace=True)
    ohlc.drop(['high', 'low'], axis=1, inplace=True)
    ohlc['EMA_6'] = ohlc['close'].ewm(span=6, adjust=False).mean()
    rsi = ta.momentum.RSIIndicator(ohlc['close'], window=14)
    ohlc['RSI'] = rsi.rsi()
    return ohlc

In [15]:
symbol1 = 'BANKNIFTY2461249300CE'
symbol2 = 'NIFTY2461322700CE'
days_before =7
timeframe = 60
ohlc = trade_option(symbol1, days_before, timeframe)
position = False
entry_price = None
total_profit_loss = 0
trades_taken = 0
stop_loss = 5
take_profit = 5
daily_pl =0
daily_pl_limit = 10

for i, row in ohlc.iterrows():
    if i == 0:
        continue

    if ohlc.iloc[i-1]['close'] > ohlc.iloc[i-1]['open'] and ohlc.iloc[i-1]['close'] > ohlc.iloc[i-1]['EMA_6']:
        if not position:
            position = True
            entry_price = row['open']
            trades_taken += 1
            print(f"Long: Entry Time: {unix_to_local_time(row['timestamp'])}, Entry Price: {entry_price}")

    if position:
        if ohlc.iloc[i-1]['close'] < ohlc.iloc[i-1]['EMA_6'] or row['open'] <= (entry_price-stop_loss) or row['open'] >= (entry_price+take_profit):
            position = False
            profit_loss = row['open'] - entry_price
            total_profit_loss += profit_loss
            daily_pl += profit_loss
            print(f"Exit Long: Time: {unix_to_local_time(row['timestamp'])}, Exit Price: {row['open']}, P/L: {profit_loss}")
            entry_price = None

print(f"\nTrades count: {trades_taken} cost: {trades_taken*60}")
print(f"\nTotal Profit/Loss: {round(total_profit_loss,1)}")
print(f"\nTake Away: {(round(total_profit_loss,1)*15)-(trades_taken*60)}")

Long: Entry Time: 2024-06-03 12:33:00, Entry Price: 2016.55
Exit Long: Time: 2024-06-03 13:35:00, Exit Price: 1998.05, P/L: -18.5
Long: Entry Time: 2024-06-04 09:16:00, Entry Price: 1290.75
Exit Long: Time: 2024-06-04 10:16:00, Exit Price: 1268.35, P/L: -22.40000000000009
Long: Entry Time: 2024-06-05 14:15:00, Entry Price: 401.0
Exit Long: Time: 2024-06-05 15:15:00, Exit Price: 586.2, P/L: 185.20000000000005
Long: Entry Time: 2024-06-06 10:15:00, Entry Price: 663.3
Exit Long: Time: 2024-06-06 11:15:00, Exit Price: 805.5, P/L: 142.20000000000005
Long: Entry Time: 2024-06-06 14:15:00, Entry Price: 605.6
Exit Long: Time: 2024-06-06 15:15:00, Exit Price: 625.0, P/L: 19.399999999999977
Long: Entry Time: 2024-06-07 10:15:00, Entry Price: 616.05
Exit Long: Time: 2024-06-07 11:15:00, Exit Price: 709.05, P/L: 93.0
Long: Entry Time: 2024-06-07 12:15:00, Entry Price: 845.85
Exit Long: Time: 2024-06-07 13:15:00, Exit Price: 825.0, P/L: -20.850000000000023

Trades count: 7 cost: 420

Total Profit/L

In [35]:
symbol1 = 'BANKNIFTY2461249300CE'
symbol2 = 'NIFTY2461322700CE'
days_before =5
timeframe = 1
ohlc = trade_option(symbol1, days_before, timeframe)
position = False
entry_price = None
total_profit_loss = 0
trades_taken = 0
stop_loss = 4
take_profit = 4
daily_pl =0

for i, row in ohlc.iterrows():
    if i == 0:
        continue

    if ohlc.iloc[i-1]['close'] > ohlc.iloc[i-1]['EMA_6'] and daily_pl < 1:
        if not position:
            position = True
            entry_price = row['open']
            trades_taken += 1
            print(f"Long: Entry Time: {unix_to_local_time(row['timestamp'])}, Entry Price: {entry_price}")

    if position:
        if ohlc.iloc[i-1]['close'] < ohlc.iloc[i-1]['EMA_6'] or row['open'] <= (entry_price-stop_loss) or row['open'] >= (entry_price+take_profit):
            position = False
            profit_loss = row['open'] - entry_price
            total_profit_loss += profit_loss
            daily_pl += profit_loss
            print(f"Exit Long: Time: {unix_to_local_time(row['timestamp'])}, Exit Price: {row['open']}, P/L: {profit_loss}")
            entry_price = None

print(f"\nTrades count: {trades_taken} cost: {trades_taken*60}")
print(f"\nTotal Profit/Loss: {round(total_profit_loss,1)}")
print(f"\nTake Away: {(round(total_profit_loss,1)*25)-(trades_taken*60)}")

Long: Entry Time: 2024-06-04 09:18:00, Entry Price: 1564.75
Exit Long: Time: 2024-06-04 09:20:00, Exit Price: 1503.75, P/L: -61.0
Long: Entry Time: 2024-06-04 09:22:00, Entry Price: 1394.35
Exit Long: Time: 2024-06-04 09:23:00, Exit Price: 1303.5, P/L: -90.84999999999991
Long: Entry Time: 2024-06-04 09:34:00, Entry Price: 1197.75
Exit Long: Time: 2024-06-04 09:35:00, Exit Price: 1236.45, P/L: 38.700000000000045
Long: Entry Time: 2024-06-04 09:36:00, Entry Price: 1250.0
Exit Long: Time: 2024-06-04 09:37:00, Exit Price: 1216.5, P/L: -33.5
Long: Entry Time: 2024-06-04 09:38:00, Entry Price: 1252.7
Exit Long: Time: 2024-06-04 09:39:00, Exit Price: 1271.15, P/L: 18.450000000000045
Long: Entry Time: 2024-06-04 09:40:00, Entry Price: 1318.85
Exit Long: Time: 2024-06-04 09:41:00, Exit Price: 1180.25, P/L: -138.5999999999999
Long: Entry Time: 2024-06-04 09:44:00, Entry Price: 1194.2
Exit Long: Time: 2024-06-04 09:45:00, Exit Price: 1139.5, P/L: -54.700000000000045
Long: Entry Time: 2024-06-04 0